In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
merge_lst_f <- 'merge.lst.v2.imp.wo.HWE.filter.tsv'
merged_bfile <- '/scratch/groups/mrivas/ukbb24983/array_imp_combined_no_cnv/pgen_v2/ukb24983_cal_hla_imp'


#### Read the ID mapping file

In [3]:
merge_lst_f %>% fread(head=F) %>% rename('pvar' = 1, 'ID_prefix' = 2) %>%
mutate(ID_map_f = str_replace(str_replace(pvar, '.zst$', ''), '.pvar$', '.IDs.tsv')) -> merge_lst_df


In [8]:
merge_lst_df %>% pull(ID_map_f) %>%
lapply(function(ID_map_f){
    fread(ID_map_f, colClasses=c(CHROM="character"))
}) %>% bind_rows() -> ID_map


#### Read the bim file from merge

In [7]:
bim_short <- fread(sprintf('%s.shortnames.bim', merged_bfile), head=F) %>%
rename('CHR' = 1, 'ID' = 2, 'CM' = 3, 'POS' = 4, 'ALT' = 5, 'REF' = 6)


#### join

In [11]:
df <- bim_short %>% mutate(sort_order = 1:n()) %>%
inner_join(ID_map %>% select(ID, full_ID), by='ID')


In [12]:
bim_short %>% dim() %>% print()
df %>% dim() %>% print()


[1] 6737150       6
[1] 6737150       8


#### write the results to `bim` file

In [13]:
df %>% arrange(sort_order) %>% 
select(CHR, full_ID, CM, POS, ALT, REF) %>%
fwrite(sprintf('%s.bim', merged_bfile), col.names=F, sep='\t')


#### check the length of the longest ID

In [14]:
df %>% select(full_ID) %>% 
mutate(ID_len = str_length(full_ID)) %>% select(ID_len) %>% max() %>% print()


[1] 181


In [16]:
df %>% select(ID) %>% 
mutate(ID_len = str_length(ID)) %>% select(ID_len) %>% max() %>% print()


[1] 10
